# BUILD FIRST MODEL
(we build here a model with no data preparation!)

### Load ML.NET libraries and initialize the ML context

In [1]:
#r "nuget:Microsoft.ML,1.4.0"
using Microsoft.ML;
using Microsoft.ML.Data;

MLContext mlContext = new MLContext(seed: 123);

Installed package Microsoft.ML version 1.4.0

### Load data models from Models.csx file 

In [1]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Models.csx"

### Load data from csv file into a dataview

In [1]:
const string TRAIN_DATASET_PATH = "./sensors_data_train.csv";
IDataView trainingData = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: TRAIN_DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

const string TEST_DATASET_PATH = "./sensors_data_test.csv";
IDataView testingData = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: TEST_DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');    

### Model builder pipeline

In [1]:
var featureColumns = new string[] { "Temperature", "Luminosity", "Infrared", "Distance" };

var trainingPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    .Append(mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy("Label", "Features"))
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

### Train the model

In [1]:
var model = trainingPipeline.Fit(trainingData);

### Evaluate the model

In [1]:
var predictions = model.Transform(testingData);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions, "Label", "Score", "PredictedLabel");

### Load formatter for metrics

In [1]:
//#r "nuget:ApexCode.Interactive.Formatting,0.0.1-beta.5"
//using ApexCode.Interactive.Formatting;
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.csx"
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;


Installed package Microsoft.Data.Analysis version 0.4.0

In [1]:

Formatters.Categories = new string[] { "FlashLight", "Infrared", "Day", "Lighter" };
Formatters.Register<ConfusionMatrix>();

### See the model quality

In [1]:
Formatters.Register<MulticlassClassificationMetrics>();
display(metrics)

EVALUATION: multi-class classification Class Value Note MacroAccuracy 0.911 the closer to 1, the better MicroAccuracy 0.928 the closer to 1, the better LogLoss 0.320 the closer to 0, the better LogLoss per Class FlashLight 0.174 the closer to 0, the better Infrared 0.658 the closer to 0, the better Day 0.261 the closer to 0, the better Lighter 0.382 the closer to 0, the better

In [1]:
display(p[@style:"font-size: 40px;"]($"MicroAccuracy: {metrics.MicroAccuracy:P2}, MacroAccuracy: {metrics.MacroAccuracy:P2}"));

MicroAccuracy: 92.83%, MacroAccuracy: 91.09%

### Predict data

In [1]:
var sampleData = new ModelInput
{
    Luminosity = 12F,
    Temperature = 22F,
    Infrared = 40F,
    Distance = 0F,
    CreatedAt = "01/03/2020 10:22:08"
};

var predictor = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
var predicted = predictor.Predict(sampleData);

In [1]:
display(p[@style:"font-size: 40px;"]($"{predicted.PredictedLabel}"));


In [1]:

#r "nuget:MathNet.Numerics"
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Helpers.csx"
display(Helpers.GetPredictionPerClass(predicted, predictor.OutputSchema));

### Save the model

In [1]:
mlContext.Model.Save(model, trainingData.Schema, "model.zip");